# Model Explainability Experimentation

In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
from data_ingestion.ingest import get_data, get_target_feature
from parameters.params import data_file_path, target_feat

<br>

## Data Ingestion

In [3]:
titanic_data = get_data(data_file_path)
titanic_data.shape

(891, 12)

In [4]:
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<br>

## Data Preprocessing

In [5]:
titanic_data['Cabin'].isnull().value_counts() # we have a lot of NaN 

True     687
False    204
Name: Cabin, dtype: int64

Cabin numbers are unique so there are not going to be very useful so we strip first letter from cabin number if available to make it categorical variable. All `NaN` are changed to `U` meaning unknown representing a category.

In [6]:
titanic_data['Cabin'] = titanic_data['Cabin'].replace(np.NaN, 'U') 
titanic_data['Cabin'] = [ln[0] for ln in titanic_data['Cabin'].values]

In [7]:
titanic_data['Cabin'].head(3)

0    U
1    C
2    U
Name: Cabin, dtype: object

Create isfemale field and use numerical values

In [8]:
titanic_data['isfemale'] = np.where(titanic_data['Sex'] == 'female', 1, 0)
titanic_data["isfemale"].head(3)

0    0
1    1
2    1
Name: isfemale, dtype: int64

Drop features not needed for model 

In [9]:
titanic_data = titanic_data[
    [f for f in list(titanic_data) if f not in [
        'Sex', 'Name', 'boat','body', 'Ticket', 'PassengerId']
    ]
]

In [10]:
titanic_data.tail(3)

,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,Embarked,isfemale
888,0,3,NaN,1,2,23.45,U,S,1
889,1,1,26.0,0,0,30.00,C,C,0
890,0,3,32.0,0,0,7.75,U,Q,0


Make Pclass actual categorical column by changing 1, 2, and 3 to first, second and third respectively.

In [11]:
titanic_data['Pclass'] = np.where(
    titanic_data['Pclass'] == 1, 'First',
    np.where(
        titanic_data['Pclass'] == 2, 'Second','Third'
        )
    )

In [12]:
titanic_data.tail(3)

,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,Embarked,isfemale
888,0,Third,NaN,1,2,23.45,U,S,1
889,1,First,26.0,0,0,30.00,C,C,0
890,0,Third,32.0,0,0,7.75,U,Q,0


For the `Embarked` feature, the are two `NaN` values, these are replaced with `U` representing unknown.

In [13]:
titanic_data['Embarked'].isnull().value_counts() # `Embarked` has 2 NaN values

False    889
True       2
Name: Embarked, dtype: int64

In [14]:
titanic_data['Embarked'] = titanic_data['Embarked'].replace(np.NaN, 'Unknown') 

Data imputation is done on the `Age` feature using the mean. i.e. fill the NaN with mean value

In [15]:
titanic_data['Age'].isnull().value_counts() # we have a lot of NaN 

False    714
True     177
Name: Age, dtype: int64

In [16]:
titanic_data['Age'] = titanic_data['Age'].fillna(titanic_data['Age'].mean())

In [17]:
titanic_data['Age'].head(3)

0    22.0
1    38.0
2    26.0
Name: Age, dtype: float64

In [18]:
titanic_data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,Embarked,isfemale
0,0,Third,22.0,1,0,7.2500,U,S,0
1,1,First,38.0,1,0,71.2833,C,C,1
2,1,Third,26.0,0,0,7.9250,U,S,1
3,1,First,35.0,1,0,53.1000,C,S,1
4,0,Third,35.0,0,0,8.0500,U,S,0


In [19]:
target = titanic_data["Survived"]
target.head(3)

0    0
1    1
2    1
Name: Survived, dtype: int64

In [20]:
features = titanic_data.drop(target_feat, axis='columns')
features.head(3)

,Pclass,Age,SibSp,Parch,Fare,Cabin,Embarked,isfemale
0,Third,22.0,1,0,7.2500,U,S,0
1,First,38.0,1,0,71.2833,C,C,1
2,Third,26.0,0,0,7.9250,U,S,1


In [21]:
# features, target = get_target_feature(titanic_data, target_feat)